In [1]:
from qiskit_aer import Aer

from qiskit.primitives import StatevectorSampler

from qiskit import QuantumCircuit


qc = QuantumCircuit(2)
qc.h(0)
qc.h(1)
qc.measure_all()


# Statevector Sampler
sampler = StatevectorSampler(seed=1)
job = sampler.run([(qc)], shots=1000)
result = job.result()[0]
result.data.meas.get_counts()

{'10': 223, '11': 270, '00': 239, '01': 268}

In [2]:
# set seed after creation (probably notneeded)
sampler._seed = 1234
job = sampler.run([(qc)], shots=1000)
result = job.result()[0]
result.data.meas.get_counts()

{'11': 242, '01': 244, '00': 254, '10': 260}

In [3]:
from qiskit.primitives import BackendSamplerV2  # Only available at Qiskit 1.2

# Seed not working?

backend = Aer.get_backend("qasm_simulator")
# this is how to set a seed for the qasm simulator, default shots can be set with default_shots
sampler = BackendSamplerV2(backend=backend, options={"seed_simulator": 1})

job = sampler.run([(qc)], shots=1000)
result = job.result()[0]
result.data.meas.get_counts()

{'10': 266, '11': 253, '00': 225, '01': 256}

In [4]:
from qiskit_ibm_runtime import SamplerV2
from qiskit_ibm_runtime.fake_provider import FakeManilaV2

# this is similar to the old optimization level of the V1 Primitives
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

fake_backend = FakeManilaV2()

# In case o the fake backend, the seed is set in the options
options = {"simulator": {"seed_simulator": 42}}

sampler = SamplerV2(backend=fake_backend, options=options)

# transpiling has to be done before running the job :/
pm = generate_preset_pass_manager(backend=fake_backend, optimization_level=1)
isa_qc = pm.run(qc)

job = sampler.run([(isa_qc)], shots=1000)
result = job.result()[0]
result.data.meas.get_counts()

{'00': 299, '10': 281, '11': 199, '01': 221}